In [1]:
import os, sys, random, gc
import numpy as np
import pandas as pd
import torch
from onnxruntime.quantization import quantize_qat,quantize_static,quantize,quantize_dynamic

sys.path.append('../../')
sys.path.append('../')

from playdict_ocr.tokenization import Tokenizer
from datasets import PartitionedTrainDataset, TrainDataset, TestDataset

In [2]:
class CFG:
    max_dec_len=25
    size=(144, 32)
    epochs, batch_size = 3, 256
    max_grad_norm=4
    embed_dim, attention_dim = 32, 192
    encoder_dim, decoder_dim = 192, 192

tokenizer = Tokenizer()

In [3]:
''.join(tokenizer.i2w)

'<PAD><START><END>ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz~$%&@0123456789* -([)]"!,.:;?'

# MODEL

In [4]:
import keras4torch as k4t
from models import EncoderDecoderModel

from encoders.repvgg import RepVGG
from decoders.rnn import DecoderWithAttention

encoder = RepVGG(
        num_blocks=[2, 4, 6],
        width_multiplier=[0.75, 0.75, 0.75],
        use_se=False, in_channels=1, output_channels=CFG.encoder_dim)

decoder = DecoderWithAttention(attention_dim=CFG.attention_dim,
                                    embed_dim=CFG.embed_dim,
                                    decoder_dim=CFG.decoder_dim,
                                    vocab_size=tokenizer.vocab_size,
                                    encoder_dim=CFG.encoder_dim,
                                    max_dec_len=CFG.max_dec_len)

model = EncoderDecoderModel(encoder, decoder)

Encoder: 2775744
Decoder: 846421


In [5]:
x = torch.randn([1,1,32,144])

In [6]:
model.deploy()

torch.onnx.export(model.cpu(),
    x, "saved_model/test.onnx",
    verbose=False,
    opset_version=11,
    input_names=['x'],
    output_names=['y'],
    do_constant_folding=False
)

In [7]:
quantize_qat("saved_model/test.onnx", "saved_model/test_quantized.onnx", per_channel=True, reduce_range=True)
os.path.getsize("saved_model/test_quantized.onnx") / 1024 / 1024

5.6432342529296875